In [ ]:
import datetime
from datetime import date

# URL of the GeoServer
owsBaseURL='http://mapcachedev.vgt.vito.be/geoserver/PV_MEP/ows'

# Username and password
credentials=('vroeyt', 'vroeyt')

# The longitude/latitude point of interest
lon,lat = 4.71213078915, 50.5839444717

# The period of interest
startDate = date(2014, 9, 1)
endDate = date(2015, 5, 1)

# The images to be processed
ndviTS_id       = 'PV_MEP__PROBAV_S5_TOC_100M_NDVI'
ndvi300TS_id    = 'PV_MEP__PROBAV_S10_TOC_333M_NDVI'
bitmaskTS_id    = 'PV_MEP__PROBAV_S5_TOC_100M_BITMASK'
bitmask300TS_id = 'PV_MEP__PROBAV_S10_TOC_333M_BITMASK'

In [ ]:
import requests
from xml.etree import ElementTree

def getCoverages():

    payload = {
        'service': 'WCS',
        'request': 'GetCapabilities',
        'version': '2.0.1',
    }
    
    response = requests.get(owsBaseURL, params=payload)
    tree = ElementTree.fromstring(response.content)
    namespaces = {
        'wcs': 'http://www.opengis.net/wcs/1.1.1',
        'wcs2': 'http://www.opengis.net/wcs/2.0'
    }
    
    coverages = map(lambda x:x.text, tree.findall('wcs2:Contents/wcs2:CoverageSummary/wcs2:CoverageId',namespaces))
    return coverages

coverages = getCoverages()
print "\n".join(coverages)

In [ ]:
import datetime
import matplotlib.pyplot as plt
from datetime import date

def pointWCSRequest(coverageId, lon, lat, startDate = None, endDate = None):
    
    wcsURL = owsBaseURL
    
    payload = {
        'SERVICE': 'WCS',
        'REQUEST': 'GetCoverage',
        'VERSION': '2.0.1',
        'coverageId': coverageId,
        'FORMAT': 'application/json',
        'subset': ['Long,http://www.opengis.net/def/crs/EPSG/0/4326('+str(lon)+')',
                   'Lat,http://www.opengis.net/def/crs/EPSG/0/4326('+str(lat)+')']
    }
        
    if startDate != None and endDate != None:
        timeSubset = 'time("'+startDate.strftime('%Y-%m-%dT%H:%M:%S.000Z')+'","'+endDate.strftime('%Y-%m-%dT%H:%M:%S.000Z')+'")'
        payload['subset'].append(timeSubset)    
    
    return requests.get(wcsURL, params=payload, auth=credentials)


def wcsTimeSeries(coverageID, lon, lat, startDate = None, endDate = None):
    response = pointWCSRequest(coverageID, lon, lat, startDate, endDate)
    timeSeries = response.json()['timeseries']
    timeSeries = map(lambda x:[datetime.datetime.fromtimestamp(x[0]/1000),x[1]], timeSeries)
    return timeSeries


def plotTimeSeries(timeSeries):
    timeAxis = map(lambda x:x[0], timeSeries)
    valueAxis = map(lambda x:x[1],timeSeries)
    plt.figure(figsize=(25,10))
    plt.plot(timeAxis, valueAxis, 'ro')

    
ndviTS       = wcsTimeSeries(ndviTS_id, lon, lat, startDate, endDate)
bitmaskTS    = wcsTimeSeries(bitmaskTS_id, lon, lat, startDate, endDate)
ndvi300TS    = wcsTimeSeries(ndvi300TS_id, lon, lat, startDate, endDate)
bitmask300TS = wcsTimeSeries(bitmask300TS_id, lon, lat, startDate, endDate)

# filter nodata values
for i, ts in enumerate(ndviTS):
    if ts[1] == 255:
        del ndviTS[i]
        del bitmaskTS[i]

In [ ]:
# Install some additional python libraries

import sys

! pip install --user mpld3

if '/home/%s/.local/lib/python2.6/site-packages/' % credentials[0] not in sys.path:
    sys.path.append('/home/%s/.local/lib/python2.6/site-packages/' % crendentials[0])
    
sys.path

In [ ]:
# Plot the data

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
fig.set_size_inches(12, 4)
ndviTimeStamps = map(lambda x:x[0], ndviTS)
ndviValues = map(lambda x:(x[1]-20)/250, ndviTS)
bitmaskValues = map(lambda x:int(x[1])&7, bitmaskTS)
print bitmaskValues

cmp = matplotlib.colors.ListedColormap(
    [ 'green'  # clear (0)
     ,'red'    # shadow (1)
     ,'orange' # unknown (2)
     ,'grey'   # cloud (3)
     ,'blue'], # ice (4)
    5)

scatter = ax.scatter(ndviTimeStamps, ndviValues, c=bitmaskValues, s=30, cmap=cmp)
ndvi300TimeStamps = map(lambda x:x[0], ndvi300TS)
ndvi300Values = map(lambda x:(x[1]-20)/250, ndvi300TS)
bitmask300Values = map(lambda x:int(x[1])&7, bitmask300TS)
scatter300 = ax.scatter(ndvi300TimeStamps, ndvi300Values,c=bitmask300Values,marker='^', s=30, cmap=cmp)

import mpld3

css = '''
    table
    {
        border-spacing: 10px;
        border-collapse: separate;
        background-color: #cccccc;
    }
    td
    {
        background-color: #cccccc;
    }
    table, td
    {
        font-family: Arial, Helvetica, sans-serif;
        text-align: left;
    }
'''

labels = ['''<table>
             <tr>
               <td>NDVI:</td>
               <td>{0}</td>
             </tr>
             <tr>
               <td>Date:</td>
               <td>{1}</td>
             </tr>             
             <tr>
               <td>Bitmask:</td>
               <td>{2}</td>
             </tr>                          
           </table>'''
          .format((ts[1]-20)/250,ts[0].strftime('%d-%m-%Y'),int(bitmaskTS[index][1])) for index, ts in enumerate(ndviTS)]

# tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
tooltip = mpld3.plugins.PointHTMLTooltip(scatter, labels=labels, css=css)
mpld3.plugins.connect(fig, tooltip)

mpld3.display(fig)